# 미션 4: 실전 RAG


## 🎯 학습 목표

이 미션에서는 실전 환경에서 활용 가능한 고급 RAG 시스템을 구현합니다.

### 주요 학습 내용
- 고급 RAG 기법 (Re-ranking, Hybrid Search 등)
- 문서 chunk 최적화
- RAG 성능 개선 기법
- 실제 서비스에 적용 가능한 RAG 파이프라인


## 📝 실습 내용

### 구현 기능
- 고급 문서 처리 (다양한 포맷 지원)
- 효율적인 chunking 전략 구현
- Hybrid Search (키워드 + 벡터 검색)
- Re-ranking을 통한 검색 품질 향상
- 답변 생성 품질 개선
- RAG 평가 지표 측정

### 사용 기술
- Python
- LangChain
- Advanced Vector Database
- Re-ranking Models
- Jupyter Notebook


### LLM
- gpt-4.1-nano
- rerank-english-v3.0

## 라이브러리 로드 & 데이터 다운로드

In [1]:
import numpy as np
import bs4
import re
import os
import pandas as pd
import time

from openai import OpenAI
from dotenv import load_dotenv

from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import Faithfulness, AnswerCorrectness, ContextRelevance, ContextRecall, ContextPrecision
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

from langchain_community.document_loaders import DirectoryLoader, TextLoader, CSVLoader
from langchain.vectorstores import FAISS

from langchain.retrievers import EnsembleRetriever 
from langchain_community.retrievers import BM25Retriever  
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain.schema import Document
from huggingface_hub import snapshot_download

In [2]:
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

In [3]:

snapshot_download(
    repo_id='allganize/rag-ko',
    repo_type='dataset',
    local_dir='./res/rag-ko',
    local_dir_use_symlinks=False
)

Returning existing local_dir `res/rag-ko` as remote repo cannot be accessed in `snapshot_download` (504 Server Error: Gateway Time-out for url: https://huggingface.co/api/datasets/allganize/rag-ko/revision/main).


'/home/user/Desktop/Camp/holiday-mission_조재훈/mission4_20251008/res/rag-ko'

In [4]:
df = pd.read_parquet('./res/rag-ko/data/test-00000-of-00001.parquet')

In [5]:
df.head(5)

,index,system,human,answer,answer_position,answer_context_title,answer_context_summary
0,0,You are a financial expert.\nYou are fluent at...,글로벌 저금리 현상이 부각된 원인은 무엇인가요?,"글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...",3,132579145651725164_KIFRT2021-02 (2),2. 뉴노멀의 도래(신 3저의 도래)\n최근 코로나 사태를 포함하여 2008년 글로...
1,1,You are a financial expert.\nYou are fluent at...,고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.,고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...,2,132579145651725164_KIFRT2021-02 (2),축소되어 자산운용의 어려움이 가중\n∙ 투자자 입장에서는 원하는 수익률을 얻기 어려...
2,2,You are a financial expert.\nYou are fluent at...,20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...,20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...,1,132579145651725164_KIFRT2021-02 (2),1. 디지털화의 진전과 금융투자업의 변화 : 20세기 이후\n가. 1950~60년대...
3,3,You are a financial expert.\nYou are fluent at...,2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?,"2000년대 이후 글로벌 IB들은 스스로 전산화를 추진하였으며, 이와 밀접하게 연관...",3,132579145651725164_KIFRT2021-02 (2),"∙ 또한, 투자은행의 명성에 의존하는 대표적인 업무인 IPO, M&A\n등도 전산화..."
4,4,You are a financial expert.\nYou are fluent at...,금융시스템 개혁법은 언제 제정되었나요?,금융시스템 개혁법은 1998년에 제정되었습니다.,3,132579145651725164_KIFRT2021-02 (2),및 일본 내 해외투자 수요를 포용하는데 적극적\n일련의 변화과정에서 일본 금융당국의...


## 고급 문서 처리

In [6]:
def parquet_system_loader(path):
    df = pd.read_parquet(path)
    return [Document(page_content=row['system']) for _, row in df.iterrows()]

def extract_contexts(system_prompt):
    context_match = re.search(r'CONTEXT="""\n(.*?)"""', system_prompt, re.DOTALL)
    if not context_match:
        return []

    context_text = context_match.group(1)

    contexts = re.findall(r'\(context \d+\)=(.*?)(?=\n\(context \d+\)=|\Z)', context_text, re.DOTALL)

    cleaned_contexts = ['']
    for context in contexts:
        lines = context.split('\n')
        cleaned_lines = [line for line in lines if not line.strip().startswith('Title:')]
        cleaned_context = '\n'.join(cleaned_lines).strip()
        cleaned_contexts.append(cleaned_context)

    return cleaned_contexts

docs = parquet_system_loader("./res/rag-ko/data/test-00000-of-00001.parquet")


In [7]:
questions = df['human'].values[:10]
answers = df['answer'].values[:10]
answers_contexts = [extract_contexts(data[1])[int(data[4])] for data in df.values[:10]]

In [8]:
docs = [Document(page_content=text)  for data in df.values for text in extract_contexts(data[1])[1:] ] 

## 효율적인 chunking 전략 구현

In [9]:
chunk_size = 100
chunk_overlap = 10

In [10]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [11]:
chunked_docs = r_splitter.split_documents(docs)

In [12]:
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
faiss_embedded = FAISS.from_documents(documents=chunked_docs, embedding=embeddings_model)

/tmp/ipykernel_1144943/872031162.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")


## Hybrid Search

In [13]:
faiss_retriver = faiss_embedded.as_retriever(search_kwargs={"k": 2})
bm25_retriver = BM25Retriever.from_documents(documents=chunked_docs, k=2)
ensemble_retriver = EnsembleRetriever(
    retrievers=[bm25_retriver, faiss_retriver], weights=[0.5, 0.5]
)

In [14]:
query = questions[0]
ensemble_result = ensemble_retriver.get_relevant_documents(query)
bm25_result = bm25_retriver.get_relevant_documents(query)
faiss_result = faiss_retriver.get_relevant_documents(query)
result = {
    'bm25' : bm25_result,
    'faiss' : faiss_result,
    'ensemble' : ensemble_result
}


/tmp/ipykernel_1144943/3554958061.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ensemble_result = ensemble_retriver.get_relevant_documents(query)


In [15]:
result

{'bm25': [Document(metadata={}, page_content='∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의\n완화적 통화정책으로 글로벌 저금리 현상이 부각'),
  Document(metadata={}, page_content='등 우리나라의 저금리 기조도 장기화될 가능성이 높아 보인다. 저금리, 저성장이 세간에 부각된 것은 2008년 글로벌 금융위기 이후 극복하')],
 'faiss': [Document(id='ac14e7d6-779a-4e7e-9557-262529c74738', metadata={}, page_content='∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의\n완화적 통화정책으로 글로벌 저금리 현상이 부각'),
  Document(id='03a75885-8c16-44cd-9b6d-5cdf192d69da', metadata={}, page_content='저금리·저성장 상황이 이어지고 있다. 글로벌 금융위기를 극복하는 과정에서\n미국, 유럽, 일본 등 주요국의 완화적 통화정책으로 글로벌 저금리 현상이')],
 'ensemble': [Document(metadata={}, page_content='∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의\n완화적 통화정책으로 글로벌 저금리 현상이 부각'),
  Document(metadata={}, page_content='등 우리나라의 저금리 기조도 장기화될 가능성이 높아 보인다. 저금리, 저성장이 세간에 부각된 것은 2008년 글로벌 금융위기 이후 극복하'),
  Document(id='03a75885-8c16-44cd-9b6d-5cdf192d69da', metadata={}, page_content='저금리·저성장 상황이 이어지고 있다. 글로벌 금융위기를 극복하는 과정에서\n미국, 유럽, 일본 등 주요국의 완화적 통화정책으로 글로벌 저금리 현상이')]}

## Re-ranking을 통한 검색 품질 향상

In [16]:
compressor = CohereRerank(model="rerank-english-v3.0")

/tmp/ipykernel_1144943/3976518952.py:1: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(model="rerank-english-v3.0")


In [17]:
query = "AI가 뭐야?"

compressor = CohereRerank(model="rerank-english-v3.0", top_n=5)
ensemble_compression_retriever = ContextualCompressionRetriever(
base_compressor=compressor, base_retriever=ensemble_retriver
)

time.sleep(5)
compressor = CohereRerank(model="rerank-english-v3.0", top_n=5)
bm25_compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=bm25_retriver
)
time.sleep(5)

compressor = CohereRerank(model="rerank-english-v3.0", top_n=5)
faiss_compression_retriever = ContextualCompressionRetriever(
    
    base_compressor=compressor, base_retriever=faiss_retriver
)
time.sleep(5)


result['bm25_compress'] = bm25_compression_retriever.get_relevant_documents(query)
time.sleep(15)

result['faiss_compress'] = faiss_compression_retriever.get_relevant_documents(query)
time.sleep(15)

result['ensemble_compress'] = ensemble_compression_retriever.get_relevant_documents(query)
time.sleep(15)


## 답변 생성 품질 개선

In [18]:
llm_rag_result = {
    'result' : [],
    'context' : [],

}

In [19]:
retriever_list = [
    faiss_retriver, 
    bm25_retriver, 
    ensemble_retriver, 
    faiss_compression_retriever, 
    bm25_compression_retriever, 
    ensemble_compression_retriever
]
name_list = [
    'faiss_retriver', 
    'bm25_retriver', 
    'ensemble_retriver', 
    'faiss_compression_retriever', 
    'bm25_compression_retriever', 
    'ensemble_compression_retriever'
]

In [20]:
for retriever in retriever_list:

    llm = ChatOpenAI(model="gpt-4.1-nano")

    qa = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        return_source_documents = True)
    predictions = []
    contexts = []
    for question in questions:
        answer = qa(question)
        time.sleep(15)
        predictions.append(answer['result'])
        contexts.append([source.page_content for source in answer['source_documents']])
    llm_rag_result['result'].append(predictions)
    llm_rag_result['context'].append(contexts)


/tmp/ipykernel_1144943/2710589527.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-nano")
/tmp/ipykernel_1144943/2710589527.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa(question)


In [21]:
for i in range(len(questions[:2])):
    print(questions[i])

    for name, result in zip(name_list, llm_rag_result['result']):
        print(f"{name} : {result[i]}")
        print()
        print('*'*15)
        print()

    
    print('-'*15)
    print('-'*15)


글로벌 저금리 현상이 부각된 원인은 무엇인가요?
faiss_retriver : 글로벌 저금리 현상이 부각된 원인은 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국들이 완화적 통화정책을 시행했기 때문입니다. 이러한 정책으로 인해 글로벌 금융 시장에 유동성이 풍부해지고, 금리 인하가 지속되면서 저금리 환경이 형성되었습니다.

***************

bm25_retriver : 글로벌 저금리 현상이 부각된 원인은 2008년 글로벌 금융위기 이후 주요국들이 경제 회복을 지원하기 위해 완화적 통화정책을 펼친 데 있으며, 이로 인해 저금리 기조가 장기화되었기 때문입니다.

***************

ensemble_retriver : 글로벌 저금리 현상이 부각된 원인은 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국이 완화적 통화정책을 시행했기 때문입니다. 이를 통해 경기 부양과 금융시장 안정화를 도모했으며, 이러한 정책들이 장기적으로 글로벌 저금리 환경을 형성하게 되었습니다.

***************

faiss_compression_retriever : 글로벌 저금리 현상이 부각된 원인은 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국들이 완화적 통화정책을 펼쳤기 때문입니다. 이러한 정책으로 인해 글로벌 금융 환경이 저금리 기조로 전환되면서 저금리·저성장 상황이 이어지고 있습니다.

***************

bm25_compression_retriever : 글로벌 저금리 현상이 부각된 원인은 2008년 글로벌 금융위기 이후 주요국들이 완화적 통화정책을 펼친 데 있습니다. 미국, 유럽, 일본 등 주요국들이 경기 부양을 위해 금리를 낮추면서 장기적으로 저금리 기조가 지속되었고, 이로 인해 글로벌 저금리 현상이 두드러지게 되었습니다.

***************

ensemble_compression_retriever : 글로벌 저금리 현상이 부각된 원인은 2008년 글로벌 금융위기 

## RAG 평가 지표 측정

In [29]:
evaluate_dict = {
    'Faithfulness' : [], 'ContextRelevance' : [], "AnswerCorrectness": [], 'ContextRecall': [], 'ContextPrecision': []
}

for predictions, contexts_predictions in zip(llm_rag_result['result'], llm_rag_result['context']):

    data_samples = {
        'question': questions,
        'answer': predictions,
        'contexts' : contexts_predictions,
        'ground_truth' : answers

    }

    dataset = Dataset.from_dict(data_samples)

    score = evaluate(dataset, metrics=[Faithfulness(),  ContextRelevance(), AnswerCorrectness(), ContextRecall(), ContextPrecision()])
    evaluate_dict['Faithfulness'].append(score['faithfulness'])
    evaluate_dict['ContextRelevance'].append(score['nv_context_relevance'])
    evaluate_dict['AnswerCorrectness'].append(score['nv_context_relevance'])
    evaluate_dict['ContextRecall'].append(score['nv_context_relevance'])
    evaluate_dict['ContextPrecision'].append(score['nv_context_relevance'])

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [38]:
from prettytable import PrettyTable
# --- 1. mean과 std 계산 ---
summary_data = {}
for metric, values in evaluate_dict.items():
    df = pd.DataFrame(values, index=name_list)
    summary_data[metric + "_mean"] = df.mean(axis=1)
    summary_data[metric + "_std"] = df.std(axis=1)

summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.round(3)

# --- 2. mean ± std 문자열로 병합 ---
merged_df = pd.DataFrame(index=name_list)
for metric in evaluate_dict.keys():
    merged_df[metric] = summary_df[metric + "_mean"].astype(str) + " ± " + summary_df[metric + "_std"].astype(str)

# --- 3. 색상 강조 처리 ---
colored_df = merged_df.copy()

for metric in evaluate_dict.keys():
    # 해당 지표의 mean 값 기준으로 정렬
    metric_mean = summary_df[metric + "_mean"]
    sorted_idx = metric_mean.sort_values(ascending=False).index

    top1, top2 = sorted_idx[0], sorted_idx[1]
    for retriever in name_list:
        value = merged_df.loc[retriever, metric]
        if retriever == top1:
            colored_df.loc[retriever, metric] = f"\033[91m{value}\033[0m"  # red
        elif retriever == top2:
            colored_df.loc[retriever, metric] = f"\033[94m{value}\033[0m"  # blue
        else:
            colored_df.loc[retriever, metric] = value

# --- 4. PrettyTable 출력 ---
table = PrettyTable()
table.field_names = ["Retriever"] + list(evaluate_dict.keys())

for retriever in name_list:
    row = [retriever] + [colored_df.loc[retriever, m] for m in evaluate_dict.keys()]
    table.add_row(row)

print(table)

+--------------------------------+---------------+------------------+-------------------+---------------+------------------+
|           Retriever            |  Faithfulness | ContextRelevance | AnswerCorrectness | ContextRecall | ContextPrecision |
+--------------------------------+---------------+------------------+-------------------+---------------+------------------+
|         faiss_retriver         | 0.777 ± 0.335 |  0.775 ± 0.343   |   0.775 ± 0.343   | 0.775 ± 0.343 |  0.775 ± 0.343   |
|         bm25_retriver          | 0.583 ± 0.375 |   0.85 ± 0.337   |    0.85 ± 0.337   |  0.85 ± 0.337 |   0.85 ± 0.337   |
|       ensemble_retriver        | 0.724 ± 0.269 |    1.0 ± 0.0     |     1.0 ± 0.0     |   1.0 ± 0.0   |    1.0 ± 0.0     |
|  faiss_compression_retriever   | 0.718 ± 0.333 |  0.775 ± 0.343   |   0.775 ± 0.343   | 0.775 ± 0.343 |  0.775 ± 0.343   |
|   bm25_compression_retriever   | 0.504 ± 0.425 |   0.85 ± 0.337   |    0.85 ± 0.337   |  0.85 ± 0.337 |   0.85 ± 0.337   |
